# Generate release notes

This notebook guides you through the process of creating release notes. Using `generate_release_objects.py` and the OpenAI API, we are able to automate the release notes authoring process. 

This notebook grabs all the release notes information from GitHub when you provide a release URL. It then does some processing to sort by labels and then put everything together. Then, we put it through OpenAI to do some editing. It still needs human editing, which you can do after you run this notebook.

After running the notebook, you'll see new generated release notes added to our Quarto docs site that you can preview and edit further. It will be under `~/site/releases`.

Please have your release URLs ready to use this notebook. You will paste them into the prompt once you run it.

## Contents<a id='toc0_'></a>    
- [Prerequisites](#toc1_)    
- [Setup](#toc2_)    
  - [Import necessary libraries](#toc2_1_)    
  - [Set up OpenAI API](#toc2_2_)    
  - [Set labels](#toc2_3_)    
  - [Collect GitHub URLs](#toc2_4_)    
  - [Set the release date](#toc2_5_)    
- [Extract PR information](#toc3_)    
  - [Create release folder](#toc3_1_)    
  - [Start writing to release notes file](#toc3_2_)    
  - [Set up release notes components](#toc3_3_)    
  - [Set the repository and tag name](#toc3_4_)    
  - [Extract PRs from each URL](#toc3_5_)    
  - [Load PR data](#toc3_6_)    
- [Edit release notes](#toc4_)    
  - [Edit the release notes body](#toc4_1_)    
  - [Load Git diff - DELETE!!!](#toc4_2_)    
  - [Use OpenAI API to interpret the Git Diff - DELETE!!!](#toc4_3_)    
  - [Compare outputs - DELETE!!!!](#toc4_4_)    
  - [Edit each title](#toc4_5_)    
  - [Set labels for each PR](#toc4_6_)    
  - [Assign PR details to PR](#toc4_7_)    
  - [Combine all PR data into the same release notes components](#toc4_8_)    
- [Add release notes to docsite and preview](#toc5_)    
  - [Write release notes to file](#toc5_1_)    
  - [Update sidebar](#toc5_2_)    
  - [Show files to commit](#toc5_3_)    
  - [Preview and edit changes](#toc5_4_)    
- [Next steps](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=4
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_'></a>Prerequisites [](#toc0_)
You should be on a separate branch associated to the story for the release notes. See [our release notes guide](https://www.notion.so/validmind/On-release-notes-20de4e7ea03f402587514f6c9eda3bb1) for the steps needed before running this notebook.

## <a id='toc2_'></a>Setup [](#toc0_)

### <a id='toc2_1_'></a>Import necessary libraries [](#toc0_)

This cell imports any dependencies and some functions from `generate_release_objects.py`.

In [1]:
import requests
import subprocess
import json
import re
import shutil
import numpy as np
import datetime
import openai
from dotenv import load_dotenv
import os

from generate_release_objects import ReleaseURL, PR
from generate_release_objects import get_release_date, write_prs_to_file, collect_github_urls

### <a id='toc2_2_'></a>Set up OpenAI API [](#toc0_)

Running this cell grabs your OpenAI API secret key from your `.env` file. If the relative path to your `.env` file is not `../.env`, change it to your relative path.

In [2]:
def setup_openai_api():
    """Loads .env file and updates the OpenAI API key. 
    
    Replace '../.env' with the relative path to your .env file.

    Modifies:
        openai.api_key
    """
    # Load environment variables
    load_dotenv('../.env') # replace to match your correct path

    # Get the OpenAI API key
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        raise EnvironmentError("OpenAI API key is not set in .env file.")

    # Set the API key for the OpenAI library
    openai.api_key = api_key

setup_openai_api()

### <a id='toc2_3_'></a>Set labels [](#toc0_)

This cell creates the main sections of the release notes. `label_hierarchy` shows the order in which updates will be shown.

In [3]:

label_to_category = {
    "highlight": "## Release highlights",
    "enhancement": "## Enhancements",
    "deprecation": "## Deprecations",
    "bug": "## Bug fixes",
    "documentation": "## Documentation"
}

categories = { 
    "highlight": [],
    "enhancement": [],
    "deprecation": [],
    "bug": [],
    "documentation": []
}

label_hierarchy = ["highlight", "deprecation", "bug", "enhancement", "documentation"]

### <a id='toc2_4_'></a>Collect GitHub URLs [](#toc0_)

Running this cell will prompt you to enter your GitHub release URLs. Keep pasting them in until you're done, then press enter again.

Example release URL: https://github.com/validmind/documentation/releases/tag/v2.4.4

In [4]:

github_urls = collect_github_urls() # the only big global variable

# TODO: print urls or tag/repo name in output as you're going

https://github.com/validmind/developer-framework/releases/tag/v2.4.10 added.

https://github.com/validmind/documentation/releases/tag/v2.4.10 added.

https://github.com/validmind/frontend/releases/tag/v1.22.12 added.



### <a id='toc2_5_'></a>Set the release date [](#toc0_)
Running this cell will prompt you to enter the desired release date. 
The default is 3 business days from today if you leave the prompt empty.

In [5]:

release_datetime = get_release_date()
formatted_release_date = release_datetime.strftime("%Y-%b-%d").lower()
original_release_date = release_datetime.strftime("%B %-d, %Y")

Release date: 2024-08-13 00:00:00



## <a id='toc3_'></a>Extract PR information [](#toc0_)

### <a id='toc3_1_'></a>Create release folder [](#toc0_)

These lines will create a folder inside of `~/site/releases` for the release notes. The folder name is the release date, as per our convention.

In [6]:

directory_path = f"../site/releases/{formatted_release_date}/"
os.makedirs(directory_path, exist_ok=True)
output_file = f"{directory_path}release-notes.qmd"
print(f"release-notes.qmd in {directory_path} created.")

release-notes.qmd in ../site/releases/2024-aug-13/ created.


### <a id='toc3_2_'></a>Start writing to release notes file [](#toc0_)
This block writes the title of the release notes into the final release notes file.

In [7]:

print("Generating & editing release notes ...")

with open(output_file, "w") as file:
    file.write(f"---\ntitle: \"{original_release_date}\"\n---\n\n")


Generating & editing release notes ...


### <a id='toc3_3_'></a>Set up release notes components [](#toc0_)
`release_components` will contain all the components of the release notes in the form of a dictionary. Later, we will merge everything together to create the release notes.

In [8]:
release_components = dict()
release_components.update(categories)
print(f"release components so far: {release_components}")

release components so far: {'highlight': [], 'enhancement': [], 'deprecation': [], 'bug': [], 'documentation': []}


### <a id='toc3_4_'></a>Set the repository and tag name [](#toc0_)
This block checks every URL and assigns its repo name, such as `documentation` or `backend`, and its tag name.

In [9]:
for url in github_urls:
    url.set_repo_and_tag_name() 

URL: https://github.com/validmind/developer-framework/releases/tag/v2.4.10
 Repo name: validmind/developer-framework
 Tage name: v2.4.10

URL: https://github.com/validmind/documentation/releases/tag/v2.4.10
 Repo name: validmind/documentation
 Tage name: v2.4.10

URL: https://github.com/validmind/frontend/releases/tag/v1.22.12
 Repo name: validmind/frontend
 Tage name: v1.22.12



### <a id='toc3_5_'></a>Extract PRs from each URL [](#toc0_)
This block gathers all the pull requests from each release URL and stores them within the URL's object data.

In [10]:
for url in github_urls:
    url.extract_prs() # initializes PR objects into a list for each URL

Extracting PRs from https://github.com/validmind/developer-framework/releases/tag/v2.4.10...

PR #141 added.

PR #143 added.

PR #146 added.

PR #148 added.

PR #149 added.

PR #152 added.

PR #140 added.

PR #144 added.

PR #151 added.

Extracting PRs from https://github.com/validmind/documentation/releases/tag/v2.4.10...

PR #253 added.

PR #251 added.

PR #257 added.

PR #260 added.

PR #259 added.

PR #262 added.

PR #264 added.

PR #265 added.

PR #263 added.

PR #261 added.

Extracting PRs from https://github.com/validmind/frontend/releases/tag/v1.22.12...

PR #812 added.

PR #821 added.

PR #822 added.

PR #827 added.

PR #828 added.

PR #850 added.

PR #861 added.

PR #862 added.

PR #864 added.

PR #866 added.

PR #868 added.

PR #869 added.

PR #878 added.

PR #879 added.

PR #883 added.

PR #884 added.

PR #835 added.

PR #839 added.

PR #840 added.

PR #860 added.

PR #863 added.

PR #867 added.

PR #875 added.

PR #876 added.

PR #880 added.

PR #881 added.

PR #882 added.

### <a id='toc3_6_'></a>Load PR data [](#toc0_)

Using the JSON data from the PRs, this block extracts and stores information into each PR's object data.

In [11]:
for url in github_urls:
    url.populate_pr_data()

Extracting data from PR #141 in validmind/developer-framework...

Extracting data from PR #143 in validmind/developer-framework...

Extracting data from PR #146 in validmind/developer-framework...

Extracting data from PR #148 in validmind/developer-framework...

Extracting data from PR #149 in validmind/developer-framework...

Extracting data from PR #152 in validmind/developer-framework...

Extracting data from PR #140 in validmind/developer-framework...

Extracting data from PR #144 in validmind/developer-framework...

Extracting data from PR #151 in validmind/developer-framework...

Extracting data from PR #253 in validmind/documentation...

Extracting data from PR #251 in validmind/documentation...

Extracting data from PR #257 in validmind/documentation...

Extracting data from PR #260 in validmind/documentation...

Extracting data from PR #259 in validmind/documentation...

Extracting data from PR #262 in validmind/documentation...

Extracting data from PR #264 in validmind/docu

## <a id='toc4_'></a>Edit release notes [](#toc0_)

### <a id='toc4_1_'></a>Edit the release notes body [](#toc0_)

(20s)
Using the prompt below, this block feeds the body of each PR to ChatGPT for editing, skipping PRs labeled as `internal`. If you find that the output is not quite right, edit the prompt and play around with it.

In [12]:
editing_instructions_body = """
    Please edit the provided technical content according to the following guidelines:

    - Use simple and neutral language in the active voice.
    - Address users directly in the second person with "you".
    - Use present tense by avoiding the use of "will".
    - Apply sentence-style capitalization to text
    - Always capitalize the first letter of text on each line.
    - Rewrite sentences that are longer than 25 words as multiple sentences.
    - Only split text across multiple lines if the text contains more than three sentences.
    - Avoid handwaving references to "it" or "this" by including the text referred to. 
    - Treat short text of less than ten words without a period at the end as a heading. 
    - Enclose any words joined by underscores in backticks (`) if they aren't already.
    - Remove exclamation marks from text.
    - Remove quotes around non-code words.
    - Remove the text "feat:" from the output
    - Maintain existing punctuation at the end of sentences.
    - Maintain all original hyperlinks for reference.
    - Preserve all comments in the format <!--- COMMENT ---> as they appear in the text.
    """

for url in github_urls:
    for pr in url.prs:
        if pr.data_json:
            print(f"Adding PR #{pr.pr_number} from {pr.repo_name} to release notes...\n") 
            if pr.extract_external_release_notes(): pr.edit_text_with_openai(False, editing_instructions_body)


Adding PR #140 from validmind/developer-framework to release notes...

Adding PR #144 from validmind/developer-framework to release notes...

Adding PR #251 from validmind/documentation to release notes...

Adding PR #853 from validmind/frontend to release notes...

Adding PR #829 from validmind/frontend to release notes...

Adding PR #874 from validmind/frontend to release notes...

Adding PR #865 from validmind/frontend to release notes...



### <a id='toc4_2_'></a>Load Git diff - DELETE!!! [](#toc0_)

Here, we will load the differences between the code for each PR, to be interpreted by ChatGPT later.

In [ ]:
for url in github_urls:
    for pr in url.prs:
        if pr.data_json: 
            pr.load_git_diff()          

### <a id='toc4_3_'></a>Use OpenAI API to interpret the Git Diff - DELETE!!! [](#toc0_)
Now, we'll have ChatGPT explain the code differences for each PR. 

If we like this version better, we can toggle the release notes to use this instead of the text generated from above.

In [ ]:
code_explain_instructions = """
I have the following git diff output from a pull request. 
Can you explain the changes in a way that is suitable for inclusion in release notes?
The explanation should summarize what was modified, added, or removed, and the purpose of these changes.
"""

for url in github_urls:
    # for pr in url.prs:
    pr = url.prs[7]
    if pr.data_json and pr.git_diff: 
        print(f"yay")
        pr.interpret_git_diff(code_explain_instructions)

In [ ]:
print(github_urls[0].prs[7].git_diff) # debugging

### <a id='toc4_4_'></a>Compare outputs - DELETE!!!! [](#toc0_)

We can take a look at the outputs of each PR and choose which version we like better. 

In [ ]:
for url in github_urls:
    for pr in url.prs:
        if pr.data_json:
            print("Git diff and code explain:\n")
            print(pr.explained_diff) 
            print("\n")
            print("Using PR body and OpenAI editing:\n")
            print(pr.edited_text)
            print("\n")

            if input("Enter 0 for Git diff, 1 for PR body") == 0: # should get replaced by ChatGPT prompt that checks for the better one
                pr.final_text = pr.explained_diff
            else:
                pr.final_text = pr.edited_text # default will be the PR body if input is empty as well

### <a id='toc4_5_'></a>Edit each title [](#toc0_)
This block does the same as above for the titles of each PR. The output below will show:
- The original PR title
- The title after some algorithmic changes
- The title after ChatGPT edits it

If you find that it's not good after editing with ChatGPT, feel free to edit the prompt below.

In [13]:
editing_instructions_title = """
    Please edit the provided technical content according to the following guidelines:

    - Use simple and neutral language in the active voice.
    - Address users directly in the second person with "you".
    - Use present tense by avoiding the use of "will".
    - Apply sentence-style capitalization to text
    - Always capitalize the first letter of text on each line.
    - Rewrite sentences that are longer than 25 words as multiple sentences.
    - Only split text across multiple lines if the text contains more than three sentences.
    - Avoid handwaving references to "it" or "this" by including the text referred to. 
    - Treat short text of less than ten words without a period at the end as a heading. 
    - Enclose any words joined by underscores in backticks (`) if they aren't already.
    - Remove exclamation marks from text.
    - Remove quotes around non-code words.
    - Remove the text "feat:" from the output
    - Maintain existing punctuation at the end of sentences.
    - Maintain all original hyperlinks for reference.
    - Preserve all comments in the format <!--- COMMENT ---> as they appear in the text.
    """
# TODO: label each print statement

for url in github_urls:
    for pr in url.prs:
        if pr.data_json: 
            print(f"Editing title for PR #{pr.pr_number} in {pr.repo_name}...\n")
            pr.title = pr.data_json['title']
            pr.clean_title(editing_instructions_title)
            print("\n")

Editing title for PR #140 in validmind/developer-framework...

 Exclude categorical and binary features from outlier tests
Exclude categorical and binary features from outlier tests
Exclude categorical and binary features from outlier tests


Editing title for PR #144 in validmind/developer-framework...

Generalize support for comparison tests
Generalize support for comparison tests
Generalize support for comparison tests


Editing title for PR #251 in validmind/documentation...

Add Private Service Connect & rework login info
Add Private Service Connect & rework login info
Add private service connect and rework login info


Editing title for PR #853 in validmind/frontend...

Ability to customize dashboard using widgets
Ability to customize dashboard using widgets
Ability to customize dashboard using widgets


Editing title for PR #829 in validmind/frontend...

Workflow approval step should allow selecting a user role as "approval group"
Workflow approval step should allow selecting a 

### <a id='toc4_6_'></a>Set labels for each PR [](#toc0_)
This block takes the label data from each PR and assigns it to the PR.

In [14]:

for url in github_urls:
    for pr in url.prs:
        if pr.data_json: 
            pr.labels = [label['name'] for label in pr.data_json['labels']]
            print(f"PR #{pr.pr_number} from {pr.repo_name}: {pr.labels}\n")

PR #140 from validmind/developer-framework: ['bug']

PR #144 from validmind/developer-framework: ['enhancement']

PR #251 from validmind/documentation: ['enhancement']

PR #853 from validmind/frontend: ['enhancement']

PR #829 from validmind/frontend: ['enhancement']

PR #874 from validmind/frontend: ['enhancement']

PR #865 from validmind/frontend: ['enhancement']



### <a id='toc4_7_'></a>Assign PR details to PR [](#toc0_)
This block compiles all the data we found earlier for each PR into one place. 

In [15]:

for url in github_urls:
    for pr in url.prs:
        if pr.data_json: 
            pr.pr_details = {
            'pr_number': pr.pr_number,
            'title': pr.cleaned_title,
            'full_title': pr.data_json['title'],
            'url': pr.data_json['url'],
            'labels': ", ".join(pr.labels),
            'notes': pr.edited_text
            }
            print(f"PR #{pr.pr_number} from {pr.repo_name} added.\n")


PR #140 from validmind/developer-framework added.

PR #144 from validmind/developer-framework added.

PR #251 from validmind/documentation added.

PR #853 from validmind/frontend added.

PR #829 from validmind/frontend added.

PR #874 from validmind/frontend added.

PR #865 from validmind/frontend added.



### <a id='toc4_8_'></a>Combine all PR data into the same release notes components [](#toc0_)
Now, we can take all the details we compiled above and append them to our final release notes components. Since we want to show features in order of importance, we sort by the priority of the label.

In [16]:

for url in github_urls:
    for pr in url.prs:
        if pr.data_json:
            print(f"Adding PR #{pr.pr_number} from {pr.repo_name}...\n")
            assigned = False 
            for priority_label in label_hierarchy:
                if priority_label in pr.labels:
                    release_components[priority_label].append(pr.pr_details)
                    assigned = True
                    break
            if not assigned:
                release_components.setdefault('other', []).append(pr.pr_details)

Adding PR #140 from validmind/developer-framework...

Adding PR #144 from validmind/developer-framework...

Adding PR #251 from validmind/documentation...

Adding PR #853 from validmind/frontend...

Adding PR #829 from validmind/frontend...

Adding PR #874 from validmind/frontend...

Adding PR #865 from validmind/frontend...



## <a id='toc5_'></a>Add release notes to docsite and preview [](#toc0_)

### <a id='toc5_1_'></a>Write release notes to file [](#toc0_)
Now that `release_components` contains everything we need for the release notes, we can write it to our release notes file.

In [17]:
# Write categorized PRs to the file
with open(output_file, "a") as file:
    write_prs_to_file(file, release_components, label_to_category)
    print(f"Release notes added to {file.name}.")


Release notes added to ../site/releases/2024-aug-13/release-notes.qmd.


### <a id='toc5_2_'></a>Update sidebar [](#toc0_)
This block will go into our `_quarto.yml` file and add the new release notes so it shows up on the sidebar of the docsite. 

In [18]:

def update_quarto_yaml(release_date):
    """Updates the _quarto.yml file to include the release notes file so it can be accessed on the website.

    Params:
        release_date - release notes use the release date as the file name.
    
    Modifies:
        _quarto.yml file
    """
    yaml_filename = "../site/_quarto.yml"
    temp_yaml_filename = "../site/_quarto_temp.yml"

    # Copy the original YAML file to a temporary file
    shutil.copyfile(yaml_filename, temp_yaml_filename)

    with open(temp_yaml_filename, 'r') as file:
        lines = file.readlines()

    # Format the release date for insertion into the YAML file
    formatted_release_date = release_date.strftime("%Y-%b-%d").lower()

    with open(yaml_filename, 'w') as file:
        add_release_content = False
        insert_index = -1

        for i, line in enumerate(lines):
            file.write(line)
            if line.strip() == "# MAKE-RELEASE-NOTES-EMBED-MARKER":
                add_release_content = True
                insert_index = i

            if add_release_content and i == insert_index:
                file.write(f'        - releases/{formatted_release_date}/release-notes.qmd\n')
                add_release_content = False

    # Remove the temporary file
    os.remove(temp_yaml_filename)
    
    print(f"Added release notes to _quarto.yml, line {insert_index + 2}")

update_quarto_yaml(release_datetime)

Added release notes to _quarto.yml, line 106


### <a id='toc5_3_'></a>Show files to commit [](#toc0_)

In [19]:
# After completing all tasks, print git status to show output files
try:
    result = subprocess.run(["git", "status", "--short"], check=True, text=True, capture_output=True)
    lines = result.stdout.split('\n')
    print("Files to commit:")
    for line in lines:
        if line.startswith((' M', '??', 'A ')):
            print(line)
except subprocess.CalledProcessError as e:
    print("Failed to run git status:", e)

Files to commit:
 M __pycache__/generate_release_objects.cpython-311.pyc
 M generate-release-notes.ipynb
 M generate_release_objects.py
 M ../site/_quarto.yml
 M ../site/_site/about/overview.html
 M ../site/_site/notebooks.zip
 M ../site/_site/notebooks/code_samples/NLP_and_LLM/foundation_models_integration_demo.html
 M ../site/_site/notebooks/code_samples/NLP_and_LLM/foundation_models_summarization_demo.html
 M ../site/_site/notebooks/code_samples/NLP_and_LLM/hugging_face_integration_demo.html
 M ../site/_site/notebooks/code_samples/NLP_and_LLM/hugging_face_summarization_demo.html
 M ../site/_site/notebooks/code_samples/NLP_and_LLM/llm_summarization_demo.html
 M ../site/_site/notebooks/code_samples/NLP_and_LLM/prompt_validation_demo.html
 M ../site/_site/notebooks/code_samples/NLP_and_LLM/rag_documentation_demo.html
 M ../site/_site/notebooks/code_samples/credit_risk/application_scorecard_demo.html
 M ../site/_site/notebooks/code_samples/custom_tests/implement_custom_tests.html
 M ../

### <a id='toc5_4_'></a>Preview and edit changes [](#toc0_)
Run this cell to preview your changes, and make edits to the release notes file you just generated. See our [internal guide](https://www.notion.so/validmind/On-release-notes-20de4e7ea03f402587514f6c9eda3bb1) on editing release notes.

In [ ]:
%%bash
cd ../site
quarto preview

Preparing to preview
[  1/196] releases/2024-aug-13/release-notes.qmd
[  2/196] tests/prompt_validation/NegativeInstruction.md
[  3/196] tests/prompt_validation/Robustness.md
[  4/196] tests/prompt_validation/Delimitation.md
[  5/196] tests/prompt_validation/Conciseness.md
[  6/196] tests/prompt_validation/Bias.md
[  7/196] tests/prompt_validation/Clarity.md
[  8/196] tests/prompt_validation/Specificity.md
[  9/196] tests/model_validation/ModelMetadata.md
[ 10/196] tests/model_validation/embeddings/EuclideanDistanceComparison.md
[ 11/196] tests/model_validation/embeddings/CosineSimilarityHeatmap.md
[ 12/196] tests/model_validation/embeddings/StabilityAnalysisTranslation.md
[ 13/196] tests/model_validation/embeddings/PCAComponentsPairwisePlots.md
[ 14/196] tests/model_validation/embeddings/StabilityAnalysisKeyword.md
[ 15/196] tests/model_validation/embeddings/StabilityAnalysis.md
[ 16/196] tests/model_validation/embeddings/DescriptiveAnalytics.md
[ 17/196] tests/model_validation/embedd


GET: /releases/2024-aug-13/release-notes.html


**When you're done with the preview, please restart the kernel.**

## <a id='toc6_'></a>Next steps [](#toc0_)

Now that you've generated, previewed, and edited the release notes, it's time to send a commit and start a PR! Make sure you're on the branch associated to the story for the release notes. Double check with our [internal guide](https://www.notion.so/validmind/On-release-notes-20de4e7ea03f402587514f6c9eda3bb1) to see if you missed anything.